<a href="https://colab.research.google.com/github/HonkinWaffles/Google-Colab-Notebooks/blob/main/Honkin_Stable_Diffusion_%2B_ESRGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion with ESRGAN Upscaling

### Repos:
* Stable Diffusion:https://github.com/CompVis/stable-diffusion

  * Optimized Fork: https://github.com/basujindal/stable-diffusion

* Real-ESRGAN: https://github.com/xinntao/Real-ESRGAN/

### Other Information:
* Model Data: v1.4


# Optimized* TXT2IMG 
Images as large as 578x1024 but at the cost of render times.

Stable Diffusion


#### Get Nvidia-SMI


In [ ]:
import subprocess
simple_nvidia_smi_display = False
nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(nvidiasmi_output)
nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(nvidiasmi_ecc_note)

Thu Sep  8 05:51:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                   0* |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Setup

In [ ]:
!pip install wget

import os, torch, gc, shutil, wget
from getpass import getpass
from pathlib import Path

root_dir='/content/'
stable_diffusion= '/content/stable-diffusion'
real_esrgan= '/content/Real-ESRGAN'

#Start of command always switch to root
os.chdir(root_dir)

#Login to GDRIVE for file access
LOGIN_GDRIVE = True
if LOGIN_GDRIVE and not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

## Drive paths for Google Colab
# Root = /content/drive/MyDrive/GOOGLE_COLAB/
# Setup Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP
# img2img Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/IMG2IMG
# txt2img Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/TXT2IMG

## Make directory structure in drive to set everything up correctly
!mkdir -p /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/IMG2IMG/INPUT /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/IMG2IMG/OUTPUT /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/TXT2IMG
!mkdir -p /content/drive/MyDrive/GOOGLE_COLAB/ESRGAN /content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/SETUP /content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/OUTPUT 

## Check if file exists in Drive if not download - 

path_to_file = '/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/Miniconda3-latest-Linux-x86_64.sh'
path = Path(path_to_file)

if path.is_file():
    shutil.copy2('/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/Miniconda3-latest-Linux-x86_64.sh', '/content/')
    print('Copying local version of Conda')
else:
    wget.download('https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh')
    print('Downloading Miniconda from internet - Consider adding a copy to `/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/`')

# Install Miniconda
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda init bash
!conda install wget diffusers basicsr facexlib gfpgan -y

# Clone Basujindal Stable-Diffusion
!git clone https://github.com/basujindal/stable-diffusion.git
!mkdir -p /content/stable-diffusion/models/ldm/stable-diffusion-v1/

## Download Stable Diffusion v1.4 Model
path_to_file = '/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/sd-v1-4-full-ema.ckpt'
path = Path(path_to_file)
if path.is_file():
    print('Copying local version of models ')
    shutil.copy2('/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/sd-v1-4-full-ema.ckpt', 'stable-diffusion/models/ldm/stable-diffusion-v1/model.ckpt')
else:
    wget.download('https://www.googleapis.com/storage/v1/b/aai-blog-files/o/sd-v1-4.ckpt?alt=media','models/ldm/stable-diffusion-v1/model.ckpt' )
    print('Downloading model from internet - Consider adding the better copy from `Huggingface` to `stable-diffusion/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/`')

!conda env update -n base -f /content/stable-diffusion/environment.yaml


# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
os.chdir(real_esrgan)
!python -m pip install -r  /content/Real-ESRGAN/requirements.txt
!python /content/Real-ESRGAN/setup.py develop
os.chdir(root_dir)

# Download Real-ESRGAN Pretrained Model
path_to_file = '/content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/SETUP/RealESRGAN_x4plus.pth'
path = Path(path_to_file)
if path.is_file():
    shutil.copy2('/content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/SETUP/RealESRGAN_x4plus.pth', '/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth')
    print('Copying local version of models')
else:
    wget.download('https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth', '/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth')
    print('Downloading Models from internet - Consider downloading and adding it to `/content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/SETUP/`')

#End of command directory check
os.chdir(root_dir)

#### TXT2IMG

In [ ]:
root_dir='/content/'
stable_diffusion= '/content/stable-diffusion'
real_esrgan= '/content/Real-ESRGAN'

#Start of command always switch to root
os.chdir(root_dir)


#@markdown Fill out the prompt and image details then run the code to generate images

#@markdown ---
PROMPT = "Evil witch vampire, long hair, ornate, elegant, amazing, intricate, extremely detailed, smooth, beautiful composition, sharp focus, illustration, concept art, digital painting, greg rutkowski, yoshitaka amano, frank frazetta, boris vallejo, sorceress, magic, fantasy, demonic, radiating magical power, extremely detailed, face details" #@param {type:'string'}
STEPS = 125 #@param {type:"slider", min:5, max:500, step:5} 
SEED = 0 #@param {type:'integer'}
NUM_ITERS = 1 #@param {type:"slider", min:1, max:100, step:1} 
HEIGHT = 704 #@param {type:"slider", min:256, max:1920, step:64}
WIDTH = 384 #@param {type:"slider", min:256, max:1920, step:64}
SCALE = 7.5 #@param {type:"slider", min:0, max:25, step:0.1}
FILE_PATH = "/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/TXT2IMG/" #@param {type:'string'}
#@markdown ---
AUTO_UPSCALE = True #@param {type:"boolean"}
OUT_SCALE = 2 #@param {type:'integer'}
FACE_ENHANCE = False #@param {type:"boolean"}
#@markdown Precision during inference (16-32) 
FP = 32 #@param {type:"slider", min:16, max:32, step:16}
#@markdown ---



# Stable diffusion - Generate txt2img
os.chdir(stable_diffusion)
if SEED is 0:
  print('Generating Image')
  !python /content/stable-diffusion/optimizedSD/optimized_txt2img.py \
  --n_samples 1 \
  --n_iter $NUM_ITERS \
  --scale $SCALE \
  --H $HEIGHT \
  --W $WIDTH \
  --ddim_steps $STEPS \
  --outdir $FILE_PATH \
  --prompt "$PROMPT"
else:
  !python /content/stable-diffusion/optimizedSD/optimized_txt2img.py \
  --n_samples 1 \
  --n_iter $NUM_ITERS \
  --scale $SCALE \
  --H $HEIGHT \
  --W $WIDTH \
  --seed $SEED \
  --ddim_steps $STEPS \
  --outdir $FILE_PATH \
  --prompt "$PROMPT"
os.chdir(root_dir)

# Real-ESRGAN - 
os.chdir(real_esrgan)

FILE_PATH = FILE_PATH + (PROMPT.replace(" ", "_"))[0:94]
print(FILE_PATH)

if AUTO_UPSCALE is True:
    if FACE_ENHANCE is True:
      print('Upscaling Images')
      !python inference_realesrgan.py \
        -n RealESRGAN_x4plus \
        -i $FILE_PATH \
        --fp$FP \
        --outscale $OUT_SCALE \
        --face_enhance 
      !rm -rf $FILE_PATH/*
      !cp /content/Real-ESRGAN/results/* $FILE_PATH/
      !rm /content/Real-ESRGAN/results/*
    else:
      print('Upscaling Images')
      !python inference_realesrgan.py \
        -n RealESRGAN_x4plus \
        -i $FILE_PATH \
        --fp$FP \
        --outscale $OUT_SCALE 
      !rm -rf $FILE_PATH/*
      !cp /content/Real-ESRGAN/results/* $FILE_PATH/
      !rm /content/Real-ESRGAN/results/*

      
#End of command directory check
os.chdir(root_dir)


# Pipeline TXT2IMG

Uses HuggingFace Pipeline for image generation, much faster.
Allows for Bul  

### NVIDIA-SMI

In [ ]:
import subprocess
simple_nvidia_smi_display = False
nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(nvidiasmi_output)
nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(nvidiasmi_ecc_note)

Thu Sep  8 08:10:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Setup

In [ ]:
# Drive paths for Google Colab
# Root = /content/drive/MyDrive/GOOGLE_COLAB/
# Setup Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP
# img2img Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/IMG2IMG
# txt2img Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/TXT2IMG

## Install Requirements 
!pip install diffusers==0.2.4
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"
!pip install wget

## Directory Variables 
root_dir='/content/'
stable_diffusion= '/content/stable-diffusion'
real_esrgan= '/content/Real-ESRGAN'

## Load Requirements
import os, torch, gc, shutil, wget
from getpass import getpass
from pathlib import Path
from huggingface_hub import notebook_login
from google.colab import output

## Login to GDRIVE for file access
LOGIN_GDRIVE = True
if LOGIN_GDRIVE and not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

## Make directory structure in drive for Stable Diff
!mkdir -p /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/IMG2IMG/INPUT /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/IMG2IMG/OUTPUT /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/TXT2IMG

## Login to Hugging Face API
path_to_file = '/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/token'
path = Path(path_to_file)

## Check for file of Hugging Face Token 
if path.is_file():
    text_file = open(path_to_file, "r")
    token = text_file.read()
    text_file.close()
    !echo $token | huggingface-cli login
else:
    print('No token found please login manually')
    output.enable_custom_widget_manager()
    notebook_login()

# Setup Real-ESRGAN for upscaling 
!git clone https://github.com/xinntao/Real-ESRGAN.git
os.chdir(real_esrgan)
!python -m pip install -r  /content/Real-ESRGAN/requirements.txt
!python /content/Real-ESRGAN/setup.py develop
os.chdir(root_dir)

# Download Real-ESRGAN Pretrained Model
path_to_file = '/content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/SETUP/RealESRGAN_x4plus.pth'
path = Path(path_to_file)
if path.is_file():
    shutil.copy2('/content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/SETUP/RealESRGAN_x4plus.pth', '/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth')
    print('Copying local version of models')
else:
    wget.download('https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth', '/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth')
    print('Downloading Models from internet - Consider downloading and adding it to `/content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/SETUP/`')

#End of command directory check
os.chdir(root_dir)


### Start Pipeline

In [ ]:
import torch
import os
import shutil 
import subprocess
from diffusers import StableDiffusionPipeline
from torch import autocast
from traitlets.config.loader import DeferredConfigString
from google.colab import files


## Defines pipeline clean 
def clean_env():
  gc.collect()
  torch.cuda.empty_cache()

## Cleans the environment before loading the pipe
clean_env()

## Starts the Stable Diffusion pipeline
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True)  
pipe = pipe.to("cuda")


### TXT2IMG

In [ ]:
import torch
import os
import shutil 
import subprocess
import subprocess as sp
from diffusers import StableDiffusionPipeline
from torch import autocast
from traitlets.config.loader import DeferredConfigString
from google.colab import files

# File System Paths 
root_dir='/content/'
stable_diffusion= '/content/stable-diffusion'
real_esrgan= '/content/Real-ESRGAN'

## Returns to the root directory
os.chdir(root_dir)

def dummy(images, **kwargs):
  return images, False

#@markdown Fill out the prompt and image details then run the code to generate images

#@markdown ---
#@markdown Either upload a list of prompts or enter a single prompt 
UPLOAD_PROMPTS_LIST = False #@param {type:"boolean"}
PROMPT = "Test12" #@param {type:'string'}
#@markdown ---
#@markdown Stable Diffusion Options
STEPS=10 #@param {type:"slider", min:5, max:500, step:5} 
SEED=0 #@param {type:'integer'}
NUM_ITERS=1 #@param {type:"slider", min:1, max:100, step:1} 
WIDTH=512 #@param {type:"slider", min:256, max:1920, step:64}
HEIGHT=512 #@param {type:"slider", min:256, max:1920, step:64}
GUIDANCE_SCALE= 7 #@param {type:"slider", min:1, max:10, step:0.5}
#SCALE=15.3 #@param {type:"slider", min:0, max:25, step:0.1}
OUTDIR="Round-1" #@param {type:'string'}
DISABLE_NSFW_FILTER = True #@param {type:"boolean"}
#@markdown ---
#@markdown  Real-ESRGAN Options
AUTO_UPSCALE = True #@param {type:"boolean"}
OUT_SCALE = 2 #@param {type:'integer'}
FACE_ENHANCE = False #@param {type:"boolean"}
#@markdown Precision during inference (16-32) 
FP = 32 #@param {type:"slider", min:16, max:32, step:16}
#@markdown ---

OUTPUT="/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/TXT2IMG/" + OUTDIR + "/"

## Disables the NSFW filter if enabled 
if DISABLE_NSFW_FILTER is True:
  pipe.safety_checker = dummy

## Setup folders for Uploads + Results 
upload_folder = 'upload'
result_folder = 'results'
if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

## Sets the prompts_split variable to user entered or parse the txt file uploaded by the user 
if UPLOAD_PROMPTS_LIST is True:
  uploaded = files.upload()
  for filename in uploaded.keys():
    dst_path = os.path.join(upload_folder, filename)
    shutil.move(filename, dst_path)
  prompts_file = open(dst_path)
  prompts_imported = prompts_file.read()
  prompts_split = prompts_imported.splitlines()
else:
  prompts_split = [PROMPT]

## Loops through each of the prompts by the number of iters set by the user 

for x in range(NUM_ITERS):
  for i in prompts_split:
    with autocast("cuda"):
      as_string = str(x)
      path = i [0:25]
      path = OUTPUT + path
      isExist = os.path.exists(path)
      if not isExist:
        os.makedirs(path)
      if SEED is 0:
        image = pipe(i, num_inference_steps=STEPS, height=HEIGHT, width=WIDTH, guidance_scale=GUIDANCE_SCALE)["sample"][0]
      else:
        image = pipe(i, num_inference_steps=STEPS, height=HEIGHT, width=WIDTH, guidance_scale=GUIDANCE_SCALE, seed=SEED)["sample"][0]
      image.save(path + '/' +"Image_" + as_string +'.png', 'PNG')         

## Upscales generated images if selected 
if AUTO_UPSCALE is True:
  path = "\"" + path + "\""
  clean_env()
  os.chdir(real_esrgan)
  if FACE_ENHANCE is True:
      print('Upscaling Images')
      !python inference_realesrgan.py \
        -n RealESRGAN_x4plus \
        -i $path \
        --fp$FP \
        --outscale $OUT_SCALE \
        --face_enhance 
      !rm -rf $path/*
      !cp /content/Real-ESRGAN/results/* $path/
      !rm /content/Real-ESRGAN/results/*
  else:
      print('Upscaling Images')
      !python inference_realesrgan.py \
        -n RealESRGAN_x4plus \
        -i $path \
        --fp$FP \
        --outscale $OUT_SCALE 
      !rm -rf $path/*
      !cp /content/Real-ESRGAN/results/* $path/
      !rm /content/Real-ESRGAN/results/*

## Returns to the root directory
os.chdir(root_dir)

0it [00:00, ?it/s]

Upscaling Images
Testing 0 Image_0
